In [1]:
import requests
import zipfile

def dame_vista(self, id, nombre_sprint, tp_sprint, ruta_habitos):

    response = requests.get(API_URL+f'sprint_habitos_graf/{id}', stream=True)
    
    if response.status_code == 200:
        with zipfile.ZipFile(response.content) as zip_file:
            zip_file.extractall()
        
        # Crear instancias de Figure y FigureCanvas con las figuras descargadas
        graf1 = plt.imread('figura1.png')
        graf2 = plt.imread('figura2.png')
        
        fig1 = plt.figure()
        fig1.imshow(graf1)
        
        fig2 = plt.figure()
        fig2.imshow(graf2)
        
        # Crear instancias de FigureCanvas con las figuras
        canvas1 = FigureCanvas(fig1)
        canvas2 = FigureCanvas(fig2)

        # Agregar los FigureCanvas a los widgets correspondientes en tu interfaz gráfica
        self.Layout_graf1.addWidget(canvas1)
        self.Layout_graf2.addWidget(canvas2)
    else:
        print("Error al descargar las figuras")
        
dame_vista()

TypeError: dame_vista() missing 5 required positional arguments: 'self', 'id', 'nombre_sprint', 'tp_sprint', and 'ruta_habitos'

In [37]:
from services.habitos import HabitoService
from database.database import Session

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from services.utiles import convertir_horas_wh_a_entero
from matplotlib.figure import Figure


DB = Session()

class DS:
    def __init__(self, db):
        self.db = db
        
    def count_consecutive_yes(self, series):
        """
        Cuenta la cantidad de dias consecutivos de 'Yes' en una serie, es decir,
        cuantos dias seguidos se ha cumplido con el habito.

        Parameters
        ----------
        series: pd.Series
            Serie de valores que se quieren contar, estos pueden ser 'Yes' o 'No'.

        Returns
        -------
        int
            Cantidad de dias consecutivos de 'Yes' en la serie.

        Examples
        --------
        >>> count_consecutive_yes(['Yes', 'Yes', 'No', 'Yes'])
        2
        >>> count_consecutive_yes(['Yes', 'Yes', 'Yes', 'Yes'])
        4
        >>> count_consecutive_yes(['Yes', 'Yes', 'Yes', 'No'])
        3
        """
        max_consecutive_yes = 0
        current_consecutive_yes = 0
        for value in series:
            if value == 1:
                current_consecutive_yes += 1
                max_consecutive_yes = max(
                    max_consecutive_yes, current_consecutive_yes)
            else:
                current_consecutive_yes = 0
        return max_consecutive_yes
    
        
        
        
    def get_stats_habitos(self, sprint_id):
            
        habito = HabitoService(self.db)
        habitos = habito.get_habitos_sprint(sprint_id)
        habitos_dict = [d.__dict__ for d in habitos]
        df = pd.DataFrame(habitos_dict)
        
        me_interesa = df.pivot(index='date', columns='habito', values='realizado').reset_index()
        me_interesa = me_interesa.rename_axis(None, axis=1)

        # Habito mayor&menor frecuencia
        frec_habit = me_interesa.eq(1).sum()
        dict = frec_habit.to_dict()
        habits_maxfrec = [i for i in list(
            dict.keys()) if dict[i] == frec_habit.max()]
        habits_minfrec = [i for i in list(
            dict.keys()) if dict[i] == frec_habit.min()]
        habits_ordenados = frec_habit.sort_values()

        # Frecuencia x dia
        new_name = me_interesa.transpose().reset_index()

        new_name.columns = range(len(new_name.columns))
        frec_dia = new_name.eq(1).sum()

        dict_d = frec_dia.to_dict()
        days_maxfrec = [i for i in list(
            dict_d.keys()) if dict_d[i] == frec_dia.max()]
        days_minfrec = [i for i in list(
            dict_d.keys()) if dict_d[i] == frec_dia.min()]

        # Maximos dias consecutivos
        consecutive_yes_count = me_interesa.apply(self.count_consecutive_yes)

        dict_r = consecutive_yes_count.to_dict()
        habits_maxrach = [i for i in list(
            dict_r.keys()) if dict_r[i] == consecutive_yes_count.max()]

        # Porcentaje de dias cumplidos
        me_interesa.eq(1).sum().sum()
        habits_porcent = me_interesa.eq(1).sum().sum() / me_interesa.size * 100

        habits_ordenados = (
            habits_ordenados.drop(['date', 'Status'])
            .index.tolist()
            )

        return f'Habitos con mayor frecuencia: \n {habits_maxfrec}', f'Habitos con menor frecuencia: \n {habits_minfrec}', f'Dia con mas habitos cumplidos: \n {days_maxfrec}', f'Dia con menor habitos cumplidos \n {days_minfrec}', f'Habitos con mayor racha \n {habits_maxrach}', f'Porcentaje total de cumplimiento: {habits_porcent}', f'{habits_ordenados}', len(habits_ordenados)
print(DS(DB).get_stats_habitos(2))

['Ejercitar Flexibilidad', 'Organizar Local', 'Rutina de cabello', 'Visualizar futuro', '💤 Dormir Temprano', 'Leer PSE', '🧘 Meditation', '✍🏽Planificar', '🍩🙅🏽\u200d♂️Sin azúcar', 'Videoterapia', 'Anotar Gastos/Ingresos ', 'Gestión de Diamantes', '🚿Café Helado', '🤳🏽Socializar por Redes Sociales Digitales', '🔥Wake Up On Fire', '💪🏽Hacer Ejercicio', 'Programar', 'Hacer ejercicios de Mr Mandíbulas', 'Rutina Facial', '💦Tomar agua en el inicio del día', '🛌🏽 Tender Cama al Levantarme ']
("Habitos con mayor frecuencia: \n ['💦Tomar agua en el inicio del día', '🛌🏽 Tender Cama al Levantarme ']", "Habitos con menor frecuencia: \n ['date', 'Ejercitar Flexibilidad', 'Organizar Local', 'Rutina de cabello', 'Status', 'Visualizar futuro', '💤 Dormir Temprano']", 'Dia con mas habitos cumplidos: \n [8]', 'Dia con menor habitos cumplidos \n [0]', "Habitos con mayor racha \n ['💦Tomar agua en el inicio del día', '🛌🏽 Tender Cama al Levantarme ']", 'Porcentaje total de cumplimiento: 35.573122529644266', "['Ejerc